In [83]:
import getpass
import os

# if not os.environ.get("OPENAI_API_KEY"):
#   os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model
from openai import api_key, base_url

# from libs.langchain.langchain import model_laboratory

# model = init_chat_model(model="Qwen/Qwen3-235B-A22B-Instruct-2507", model_provider="openai", base_url="https://api.siliconflow.cn/v1", api_key="sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
model = init_chat_model(model="GPT-4o", model_provider="openai", base_url="https://api.deepbricks.ai/v1/", api_key="sk-C3gsaWJaOsmjrNv1fIobc4zEEsEkD15T98j80Vc8mNuSE420")
print(model)


client=<openai.resources.chat.completions.completions.Completions object at 0x7f66ee1859c0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7f66ee186080> root_client=<openai.OpenAI object at 0x7f66ee37ac80> root_async_client=<openai.AsyncOpenAI object at 0x7f66ee184370> model_name='GPT-4o' model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.deepbricks.ai/v1/'


In [84]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 25, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'thoughts_token_count': 0}, 'model_name': 'gpt-4o', 'system_fingerprint': None, 'id': 'chatcmpl-G7oYKJyb8z77D3162q380VcDroPdh3A0', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--da0c6367-260e-465b-9ae0-a6a9f9d0a6d1-0', usage_metadata={'input_tokens': 25, 'output_tokens': 5, 'total_tokens': 30, 'input_token_details': {}, 'output_token_details': {}})

In [85]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}


In [86]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

{
  "Language Type": "The user's input is pure English",
  "Your Reasoning": "The user introduced themselves in English, so the output language should be English.",
  "Your Output": "Introducing myself as Bob"
}


In [87]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

```json
{
  "Language Type": "The user's input is pure English",
  "Your Reasoning": "The user is directly asking about their own name in English.",
  "Your Output": "Asking me about your name☺️"
}
```


In [88]:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don’t know your name yet — you haven’t told me. Would you like to share it?


In [89]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [90]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [91]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
print(input_messages)
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

[HumanMessage(content="Hi! I'm Jim.", additional_kwargs={}, response_metadata={})]
================================== Ai Message ==================================

Ahoy, Jim! ☠️ Welcome aboard, matey! What seas be ye sailin’ today? 🏴‍☠️


In [92]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, ye be Jim, the bold sailor who just boarded me ship! 🏴‍☠️⚓


In [93]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [94]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [95]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! Encantado de conocerte. ¿Cómo estás hoy?


In [96]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
from transformers import AutoTokenizer

import tiktoken

# 加载Qwen模型的分词器
tokenizer = AutoTokenizer.from_pretrained(
    "GPT-4o",
    trust_remote_code=True
)


tokenizer = tiktoken.get_encoding("o200k_base")



def count_tokens_for_qwen(messages):
    """计算Qwen模型消息的token数量"""
    # 转换消息为Qwen所需的格式
    formatted_messages = []
    for msg in messages:
        if isinstance(msg, SystemMessage):
            formatted_messages.append({"role": "system", "content": msg.content})
        elif isinstance(msg, HumanMessage):
            formatted_messages.append({"role": "user", "content": msg.content})
        elif isinstance(msg, AIMessage):
            formatted_messages.append({"role": "assistant", "content": msg.content})

    # 使用分词器的chat模板处理消息
    text = tokenizer.apply_chat_template(
        formatted_messages,
        tokenize=False,
        add_generation_prompt=False
    )

    # 计算token数量
    tokens = tokenizer.encode(text)
    return len(tokens)

def custom_trim_messages(messages, max_tokens, strategy="last", start_on="human"):
    """
    自定义消息截断函数，模拟langchain的trim_messages功能
    当消息总token数超过max_tokens时进行截断
    """
    if strategy != "last":
        raise NotImplementedError("目前只实现了'last'策略")

    # 计算当前所有消息的token数
    current_tokens = count_tokens_for_qwen(messages)

    # 如果没有超过最大token数，直接返回
    if current_tokens <= max_tokens:
        return messages.copy()

    # 从最后开始逐步移除消息，直到总token数符合要求
    trimmed_messages = messages.copy()

    # 确定从哪种角色开始移除
    start_index = len(trimmed_messages) - 1

    # 如果指定了start_on，找到最后一个符合条件的消息开始移除
    if start_on == "human":
        for i in range(len(trimmed_messages)-1, -1, -1):
            if isinstance(trimmed_messages[i], HumanMessage):
                start_index = i
                break
    elif start_on == "ai":
        for i in range(len(trimmed_messages)-1, -1, -1):
            if isinstance(trimmed_messages[i], AIMessage):
                start_index = i
                break

    # 从start_index开始向前移除消息，直到满足token限制
    for i in range(start_index, -1, -1):
        # 移除当前消息
        del trimmed_messages[i]

        # 检查是否满足token限制
        if count_tokens_for_qwen(trimmed_messages) <= max_tokens:
            break

    return trimmed_messages

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer = custom_trim_messages(
    messages=messages,
    max_tokens=256,
    strategy="last",
    start_on="human"
)
print("截断后的消息:")
for msg in trimmer:
    print(f"{type(msg).__name__}: {msg.content[:30]}...")

print(f"\n原始消息token数: {count_tokens_for_qwen(messages)}")
print(f"截断后消息token数: {count_tokens_for_qwen(trimmer)}")

OSError: GPT-4o is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`